In [1]:
# Importa le librerie
import math
import random 
import statistics
import pandas as pd
import numpy as np
from pandas import read_csv
from pandas import set_option
from numpy import mean
from numpy import std
from numpy import arange
from pandas import read_csv
from sklearn.feature_selection import VarianceThreshold
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers.core import Dense, Activation, Dropout
import tensorflow as tf
from keras.callbacks import Callback
from keras import backend
from keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from math import floor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from numpy import sqrt
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import PowerTransformer
from numpy.random import seed
#import talib
#rom talib.abstract import *
import warnings
warnings.filterwarnings('ignore')
from tensorflow import keras
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.preprocessing import MinMaxScaler

import datetime
from datetime import datetime, timezone, date, timedelta
import time

import tweepy

import coinbasepro as cbp
import requests

from textblob import TextBlob
import re
import string

from numpy import mean
from numpy import median

import csv

import json

from pytrends.request import TrendReq



**Test tensorflow**

In [ ]:
""" msg = tf.constant('Hello, TensorFlow!')
tf.print(msg) """

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [2]:
# Stampa la versione di TensorFlow in uso
print(tf.__version__)

2.9.0


**load dataset**

In [3]:
df = pd.read_csv('data/bitcoin.csv', index_col='time')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 361 entries, 2022-12-20 to 2021-12-25
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   low              361 non-null    float64
 1   high             361 non-null    float64
 2   open             361 non-null    float64
 3   close            361 non-null    float64
 4   volume           361 non-null    float64
 5   greed_and_fear   361 non-null    float64
 6   google_trend     361 non-null    float64
 7   difficulty       361 non-null    float64
 8   sentiment_medio  361 non-null    float64
dtypes: float64(9)
memory usage: 28.2+ KB


In [5]:
print(df.shape)
df.head()

(361, 9)


,low,high,open,close,volume,greed_and_fear,google_trend,difficulty,sentiment_medio
time,,,,,,,,,
2022-12-20,16398.22,17060.86,16439.98,16897.65,34330.282352,29.0,23.0,3.536407e+13,0.084011
2022-12-19,16273.40,16822.84,16742.33,16439.74,26856.085987,29.0,22.0,3.536407e+13,0.068184
2022-12-18,16663.76,16875.00,16782.23,16741.16,11073.438862,26.0,19.0,3.536407e+13,0.101187
2022-12-17,16585.64,16799.99,16634.29,16782.25,18446.683602,28.0,21.0,3.460660e+13,0.053202
2022-12-16,16529.53,17525.00,17359.10,16632.64,47159.863007,29.0,27.0,3.424433e+13,0.072307


In [6]:
null = df.isnull().sum()
duplicate = df.duplicated().sum()
print(null)
print(r'valori doppi: ', duplicate)

low                0
high               0
open               0
close              0
volume             0
greed_and_fear     0
google_trend       0
difficulty         0
sentiment_medio    0
dtype: int64
valori doppi:  0


**Data preparation**

In [26]:
def split_sequences(sequence_x, sequence_y, n_steps):
    X, y = [], []
    for i in range(0,len(sequence_x),n_steps):
        X.append(sequence_x[i:i+n_steps])
        y.append(sequence_y[i:i+n_steps])
    X = np.array(X)
    y = np.array(y)
    return X,y

In [27]:
# Usa il dataframe per ottenere gli array di input e output
X = df.drop(columns=['close']) # tutte le colonne tranne 'close' sono gli input
y = df['close'] # la colonna 'close' è l'output

# Divide il dataset in set di addestramento e test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Standardizza le features con MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

**Model preparation and compilation**

In [24]:
X_test.shape

(73, 8)

In [28]:
# Costruisci il modello di deep learning
step_size = 8
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.1))
# model.add(LSTM(64))
# model.add(Dropout(0.1))
model.add(Dense(1))
model.add(Activation('linear'))

# Compila il modello
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()])
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_7 (LSTM)               (None, 128)               66560     
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
 activation_4 (Activation)   (None, 1)                 0         
                                                                 
Total params: 66,689
Trainable params: 66,689
Non-trainable params: 0
_________________________________________________________________


**Model training**

In [31]:
# Addestra il modello sui dati di addestramento
model.fit(X_train, y_train, epochs=1000, batch_size=25, verbose=2, shuffle=False)


Epoch 1/1000
12/12 - 0s - loss: 950038976.0000 - root_mean_squared_error: 30822.7031 - 122ms/epoch - 10ms/step
Epoch 2/1000
12/12 - 0s - loss: 949937984.0000 - root_mean_squared_error: 30821.0645 - 99ms/epoch - 8ms/step
Epoch 3/1000
12/12 - 0s - loss: 949838208.0000 - root_mean_squared_error: 30819.4473 - 98ms/epoch - 8ms/step
Epoch 4/1000
12/12 - 0s - loss: 949739648.0000 - root_mean_squared_error: 30817.8457 - 96ms/epoch - 8ms/step
Epoch 5/1000
12/12 - 0s - loss: 949641856.0000 - root_mean_squared_error: 30816.2617 - 98ms/epoch - 8ms/step
Epoch 6/1000
12/12 - 0s - loss: 949544960.0000 - root_mean_squared_error: 30814.6895 - 96ms/epoch - 8ms/step
Epoch 7/1000
12/12 - 0s - loss: 949448640.0000 - root_mean_squared_error: 30813.1250 - 100ms/epoch - 8ms/step
Epoch 8/1000
12/12 - 0s - loss: 949352768.0000 - root_mean_squared_error: 30811.5703 - 99ms/epoch - 8ms/step
Epoch 9/1000
12/12 - 0s - loss: 949257536.0000 - root_mean_squared_error: 30810.0234 - 95ms/epoch - 8ms/step
Epoch 10/1000
12

In [32]:
# Valuta il modello utilizzando il set di test
model.summary()

predict = model.predict(X_test, verbose=0)

metrics = model.evaluate(X_test, y_test, verbose=2)
mae = mean_absolute_error(y_test, predict)
rmse = sqrt(mean_squared_error(y_test, predict))
print('MAE before inverse Scaling: %f' % mae)
print('RMSE before inverse Scaling: %f' % rmse)


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_7 (LSTM)               (None, 128)               66560     
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
 activation_4 (Activation)   (None, 1)                 0         
                                                                 
Total params: 66,689
Trainable params: 66,689
Non-trainable params: 0
_________________________________________________________________
3/3 - 0s - loss: 832130432.0000 - root_mean_squared_error: 28846.6719 - 25ms/epoch - 8ms/step
MAE before inverse Scaling: 26766.606517
RMSE before inverse Scaling: 28846.672454


In [33]:
# Ottieni la data di oggi
today = datetime.now().date()

# Aggiungi un giorno alla data di oggi
tomorrow = today + timedelta(days=1)

# Crea un dizionario vuoto con i dati di domani
data_tomorrow = {}

# Aggiungi la data di domani al dizionario
data_tomorrow['time'] = tomorrow

# Aggiungi le altre colonne vuote al dizionario
data_tomorrow['low'] = 0
data_tomorrow['high'] = 0
data_tomorrow['open'] = 0
data_tomorrow['close'] = 0
data_tomorrow['volume'] = 0
data_tomorrow['greed_and_fear'] = 0
data_tomorrow['google_trend'] = 0
data_tomorrow['difficulty'] = 0
data_tomorrow['sentiment_medio'] = 0

# Crea un dataframe con i dati di domani
df_tomorrow = pd.DataFrame(data_tomorrow, index=[0])

df = df.reset_index()

# Unisci il dataframe di domani al dataframe storico
df = pd.concat([df_tomorrow,df.loc[:]]).reset_index(drop=True)

df = df.set_index('time')

df = df.replace(0, np.nan)
df.head()

,low,high,open,close,volume,greed_and_fear,google_trend,difficulty,sentiment_medio
time,,,,,,,,,
2022-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-20,16398.22,17060.86,16439.98,16897.65,34330.282352,29.0,23.0,3.536407e+13,0.084011
2022-12-19,16273.40,16822.84,16742.33,16439.74,26856.085987,29.0,22.0,3.536407e+13,0.068184
2022-12-18,16663.76,16875.00,16782.23,16741.16,11073.438862,26.0,19.0,3.536407e+13,0.101187
2022-12-17,16585.64,16799.99,16634.29,16782.25,18446.683602,28.0,21.0,3.460660e+13,0.053202


In [ ]:
df.info()

In [34]:
X = df.drop(columns=['close']) # tutte le colonne tranne 'close' sono gli input

In [35]:
tomorrow = X[:1]

In [36]:
tomorrow.head()

,low,high,open,volume,greed_and_fear,google_trend,difficulty,sentiment_medio
time,,,,,,,,
2022-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
# Usa il modello addestrato per fare previsioni sui dati di domani
next_day_predictions = model.predict(tomorrow)

print(next_day_predictions)

1/1 [==============================] - 1s 911ms/step
[[nan]]


**Scarico i dati di oggi**

In [ ]:
#prezzo bictoin
client = cbp.PublicClient()
candles = client.get_product_historic_rates(product_id="BTC-USD", start="2022-01-11T00:00:00", stop="2022-06-10T00:00:00", granularity=86400)
df_domani = pd.DataFrame(candles)

print('dataset set 1:', df_domani.head(2))

# Inizializza la lista data_list
data_list = []

# Effettua 31 chiamate all'API di Alternative.me
for i in range(13):
  # Prepara la richiesta HTTP
  url = "https://api.alternative.me/fng/?limit=13&timestamp=" + str(i)
  headers = {'Content-Type': 'application/json'}

  # Effettua la richiesta HTTP
  response = requests.get(url, headers=headers)

  # Carica i dati in formato JSON
  data = response.json()

  # Estrai il valore "value" e il timestamp
  value = data["data"][i]["value"]
  timestamp = data["data"][i]["timestamp"]

  # Aggiungi i valori "value" e "timestamp" nella lista data_list
  data_list.append((value, timestamp))

  # Per ogni tupla nella lista data_list
for value, timestamp in data_list:
  # Converti il timestamp in una data
  date = datetime.fromtimestamp(int(timestamp)).strftime('%Y-%m-%d')
  #date = pd.to_datetime(date)
  # Seleziona la riga del dataframe corrispondente alla data
  df_domani.loc[df_domani['time'] == date, 'greed_and_fear'] = value

print('dataset set 2:', df_domani.head(2))

  # Carica il file JSON in un oggetto python
with open('data/difficulty.json', 'r') as file:
    data = json.load(file)

# Normalizza i dati del file JSON
df_difficulty = pd.json_normalize(data, 'difficulty', ['metric1', 'metric2'])

# Crea una lista di date partendo dal 23 dicembre 2021 fino al 22 dicembre 2022
dates = pd.date_range(start='2021-12-25', end='2022-12-23', freq='D')

# Crea una nuova colonna "date" nel tuo dataframe e assegna ad ogni riga una data
df_difficulty['date'] = dates

df_difficulty = df_difficulty.drop(['x', 'metric1', 'metric2'], axis=1)

df_difficulty.rename(columns={'date': 'time'}, inplace=True)

#trasforma df['time'] in datetime
df_difficulty['time'] = pd.to_datetime(df_difficulty['time'])

#seleziono solo la data di domani
#df_difficulty = df_difficulty.loc['2022-12-21':'2022-12-12']

df_difficulty = df_difficulty.iloc[::-1]

df_domani = pd.merge(df_domani, df_difficulty, on='time')

df_domani.rename(columns={'y': 'difficulty'}, inplace=True)

print('dataset set 3:', df_domani.head(2))

#istanzio l'auth
bearer_token = "AAAAAAAAAAAAAAAAAAAAAPgukgEAAAAADYQXq5tFdfahfgxE%2FSQqxcV4OFU%3DIETAxHE05aQI4saJCxFJMQiaurCSZgYMSHZ9kl7SMVNNxM5V4S"  # BEARER_TOKEN
auth = tweepy.Client(bearer_token)
api = tweepy.API(auth)

#scarico tweets di oggi

# Numero di giorni da scaricare
num_days = 13

# Numero di tweet da scaricare ogni giorno
num_tweets = 500

# Inizializza una lista che conterrà i tweet scaricati
tweet_list = []

# Inizializza la data di inizio del loop
start_date = today = datetime.today() - timedelta(days=14)
#start_date = datetime.now() - timedelta(days=30)

# Crea il loop
for i in range(num_days):
    # Aggiorna la data di fine
    end_date = start_date + timedelta(days=1)

    # Formatta le date come stringhe nel formato richiesto dall'API
    formatted_start_date = start_date.strftime("%Y-%m-%dT%H:%M:%S")
    formatted_end_date = end_date.strftime("%Y-%m-%dT%H:%M:%S")

    #hashtag
    query= "bitcoin"

    # Scarica i tweet
    response = auth.search_all_tweets(query, max_results=num_tweets, tweet_fields=["created_at", "lang"], start_time=start_date, end_time=end_date)
    tweets = response.data

    # Aggiungi i tweet alla lista
    for tweet in tweets:
        tweet_list.append([tweet.created_at, tweet.id, tweet.text, tweet.lang])

    # Aggiorna la data di inizio
    start_date = end_date
    time.sleep(5)

df_sentiment = pd.DataFrame(data=tweet_list, columns=['created_at', 'tweet_id', 'text', 'language'])  
df_sentiment.set_index("created_at", inplace = True)

df_sentiment = df_sentiment.iloc[::-1]

def preprocess_tweet(tweet):
    # Rimuove i caratteri di nuova riga e a capo
    tweet = tweet.replace("\n", " ").replace("\r", " ")
    
    # Rimuove i link
    tweet = re.sub(r"http\S+", "", tweet)
    
    # Rimuove gli hashtag
    tweet = re.sub(r"#\S+", "", tweet)
    
    # Rimuove le menzioni
    tweet = re.sub(r"@\S+", "", tweet)
    
    # Rimuove i caratteri di punteggiatura
    tweet = tweet.translate(str.maketrans("", "", string.punctuation))
    
    # Converti il testo in minuscolo
    tweet = tweet.lower()
    
    return tweet

# Crea una copia della colonna 'text' del dataframe
df_sentiment['text_clean'] = df_sentiment['text'].copy()

# Sostituisci i testi elaborati nella colonna 'text_clean' del dataframe
df_sentiment['text_clean'] = df_sentiment['text_clean'].apply(preprocess_tweet)

def get_sentiment(tweet_text):
    analysis = TextBlob(tweet_text)
    return analysis.sentiment.polarity

df_sentiment['sentiment'] = df_sentiment['text_clean'].apply(get_sentiment)

df_sentiment = df_sentiment.reset_index()
df_sentiment['created_at'] = pd.to_datetime(df_sentiment['created_at'])

# Crea una nuova colonna 'date' che estrae la data dalla colonna 'created_at'
df_sentiment['date'] = df_sentiment['created_at'].dt.date

# raggruppa i dati per data utilizzando la colonna 'data' come chiave di raggruppamento
grouped_data = df_sentiment.groupby('date')

# per ogni gruppo di dati, calcola la mediana del sentiment e aggiungi il valore alla lista medie
medie = []
for name, group in grouped_data:
    medie.append(mean(group['sentiment']))

# crea il dizionario con le date come chiavi e i valori medi come valori
dati = {key: value for key, value in zip(grouped_data.groups.keys(), medie)}

# crea il DataFrame utilizzando il dizionario appena creato
df_medie = pd.DataFrame.from_dict(dati, orient='index')

# rinomina la colonna del DataFrame con il nome desiderato
df_medie.rename(columns={0: 'sentiment_medio'}, inplace=True)

df_medie = df_medie.reset_index()

df_medie.rename(columns={'index': 'time'}, inplace=True)

df_medie = df_medie.iloc[::-1]

#trasformo la colonna created_at in datetime
df_medie['time'] = pd.to_datetime(df_medie['time'])

#trasforma df['time'] in datetime
df_domani['time'] = pd.to_datetime(df_domani['time'])

# Unisci i due dataframe utilizzando la colonna "time" come chiave di unione
df_domani = pd.merge(df_domani, df_medie, on='time')

print('dataset set 4:', df_domani.head(2))

#scarico google trends
df_google = pd.read_csv('data/multiTimeline.csv')
df_google['time'] = pd.to_datetime(df_google['time'])
df_domani = pd.merge(df_domani, df_google, on='time')
df_domani.tail()

**unisco i nuovi dati al database originale**

In [ ]:
""" df = df.reset_index()
df_domani = df_domani.reset_index()

# Unisci il dataframe di domani al dataframe storico
df = pd.concat([df_domani,df.loc[:]]).reset_index(drop=True)

df = df.set_index('time') """


In [ ]:
df = df.set_index('time')
df.to_csv('data/bitcoin.csv')

**Preparo i nuovi dati per la previsione**

In [ ]:
""" Usa il modello addestrato per fare previsioni sui dati di domani
next_day_predictions = model.predict(domani)

print(next_day_predictions) """

**Scarico i nuovi google trends**

In [ ]:
df_google = pd.read_csv('data/multiTimeline.csv')
df = df.reset_index()
df['time'] = pd.to_datetime(df['time'])
df_google['time'] = pd.to_datetime(df_google['time'])
df_2 = pd.merge(df, df_google, on='time')
df_2 = df_2.drop(['index'], axis=1)
df_2 = df_2.set_index('time')
df_2.tail()